In [89]:
import pandas as pd
#pd.set_option('display.max_rows', None)
import re
import json
import csv

# Join Wiki Files Together

We will read in Libby and Getnet's data, join it together, export as a CSV

In [90]:
full_wards = []

with open("../data/wiki/libbys_scraped_data.json", "r") as f:
    df = json.load(f)
    for row in df:
        full_wards.append(row)
        
with open("../data/wiki/getnet_data.json", "r") as f:
    df = json.load(f)
    for row in df:
        full_wards.append(row)
        
all_alderpeople = pd.DataFrame(full_wards)

## Clean Data
### Step 1
1) Create new columns and convert specific columns to numbers

In [91]:
all_alderpeople["Clean Ward"] = all_alderpeople["Ward"].str.extract('(\\d+)')
all_alderpeople["Start Year"] = all_alderpeople["Start Date"].str.extract('(\\d{4})')
all_alderpeople["End Year"] =  all_alderpeople["End Date"].str.extract('(\\d{4})')
all_alderpeople["Start Year"] =  pd.to_numeric(all_alderpeople['Start Year'])
all_alderpeople["End Year"] =  pd.to_numeric(all_alderpeople['End Year'])


### Step 2
2) Create an "End Year for Fill" column that will include every single year someone was in office

In [92]:
all_alderpeople["End Year for Fill"] =  all_alderpeople["End Year"]
all_alderpeople["End Year for Fill"] =  all_alderpeople["End Year for Fill"].fillna(2023)
aldermen_year_check = all_alderpeople[0:1]

# for year in between Start Year and End Year for Fill
# add a column called year in office
# add row with same value but for every year served

for index, row in all_alderpeople.iterrows():
    start = int(row["Start Year"])
    end = int(row["End Year for Fill"])
    for year in range(start, end + 1): 
        new_row = pd.DataFrame([{'Ward': row["Ward"], \
            'Alderperson': row["Alderperson"], \
            'Start Date': row["Start Date"],
            'End Date': row["End Date"],
            'Party': row["Party"],
            'Notes': row["Notes"],
            'Clean Ward': int(row["Clean Ward"]),
            'Start Year': row["Start Year"],
            'End Year': row["End Year"],
            'End Year for Fill': int(year),
            }])
        
        aldermen_year_check = pd.concat([aldermen_year_check, new_row], ignore_index=True)
        
aldermen_year_check = aldermen_year_check[1::]
aldermen_year_check.reset_index(drop=True)


,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill
0,5,Paul Howard Douglas,1939,1942,None,None,5,1939,1942.0,1939.0
1,5,Paul Howard Douglas,1939,1942,None,None,5,1939,1942.0,1940.0
2,5,Paul Howard Douglas,1939,1942,None,None,5,1939,1942.0,1941.0
3,5,Paul Howard Douglas,1939,1942,None,None,5,1939,1942.0,1942.0
4,5,Leon Despres,1955,1975,None,None,5,1955,1975.0,1955.0
...,...,...,...,...,...,...,...,...,...,...
2674,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2019.0
2675,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0
2676,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0
2677,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0


### Step 3
3) Drop rows that include a year of service before 2018.

In [93]:
alders_2018_2023 =  aldermen_year_check.loc[(aldermen_year_check['End Year for Fill'] >= 2018) ]

alders_2018_2023.reset_index(drop= True)
alders_2018_2023.drop_duplicates(ignore_index= True)

alders_2018_2023

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill
45,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2018.0
46,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2019.0
47,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2020.0
48,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2021.0
49,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2022.0
...,...,...,...,...,...,...,...,...,...,...
2675,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2019.0
2676,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0
2677,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0
2678,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0


### Step 4:

Check if every ward is listed for every year

Can I rewrite the following checks to be a group by and fill from year to year

In [94]:
check_max = alders_2018_2023.groupby('Clean Ward').max('End Year for Fill')
check_max.loc[(check_max['End Year for Fill'] < 2023) ]

,Start Year,End Year,End Year for Fill
Clean Ward,,,
24,2015,2022.0,2022.0


In [95]:
check_min = alders_2018_2023.groupby('Clean Ward').min('End Year for Fill')
check_min.loc[(check_min['End Year for Fill'] > 2018) ]

,Start Year,End Year,End Year for Fill
Clean Ward,,,


### Step 4: Correction
We discover someone is missing in Ward 24 and we manually add them

In [96]:
ward_24_corection1 = pd.DataFrame([{'Ward': 24, \
            'Alderperson': "Monique Scott", \
            'Start Date': "June 22, 2022",
            'End Date': "present",
            'Party': "",
            'Notes': "",
            'Clean Ward': 24,
            'Start Year': 2022,
            'End Year': None,
            'End Year for Fill': float(2022),
            }])

ward_24_corection2 = pd.DataFrame([{'Ward': "24", \
            'Alderperson': "Monique Scott", \
            'Start Date': "June 22, 2022",
            'End Date': "present",
            'Party': "",
            'Notes': "",
            'Clean Ward': 24,
            'Start Year': "2022",
            'End Year': None,
            'End Year for Fill': float(2023),
            }])

alders_2018_2023_test = pd.concat([alders_2018_2023, ward_24_corection1, ward_24_corection2], ignore_index=True)

alders_2018_2023_test.sort_values(["Clean Ward", "End Year for Fill"])

/var/folders/_d/481kb3lx6j307ch206pvy4w40000gn/T/ipykernel_73823/1392196364.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  alders_2018_2023_test = pd.concat([alders_2018_2023, ward_24_corection1, ward_24_corection2], ignore_index=True)


,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill
220,1st,Proco Joe Moreno,"March 26, 2010","May 20, 2019",Democratic,,1,2010,2019.0,2018.0
221,1st,Proco Joe Moreno,"March 26, 2010","May 20, 2019",Democratic,,1,2010,2019.0,2019.0
222,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2019.0
223,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2020.0
224,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2021.0
...,...,...,...,...,...,...,...,...,...,...
323,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2019.0
324,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0
325,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0
326,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0


### Step 5
5) Get rid of years that have two people listed. Take the second alderperson's name

In [97]:
year_count = alders_2018_2023_test.groupby(["Clean Ward","End Year for Fill"]).size().rename("count_by_year").reset_index()

#year_count.groupby("count_by_year").size()

year_count_over = year_count.loc[(year_count['count_by_year'] > 1)]
year_count_one = year_count.loc[(year_count['count_by_year'] == 1)]


#year_count_over.sort_values(["Clean Ward", "End Year for Fill"])

year_count # 301 rows

,Clean Ward,End Year for Fill,count_by_year
0,1,2018.0,1
1,1,2019.0,2
2,1,2020.0,1
3,1,2021.0,1
4,1,2022.0,1
...,...,...,...
295,50,2019.0,1
296,50,2020.0,1
297,50,2021.0,1
298,50,2022.0,1


### Step 6
6. Concatenate data where there was just a single match to a cleaned up version of our data with 2 people for one year

6a. Create our "anti-join" dataset, wards who had just a single match

In [98]:
one_entry = pd.merge(alders_2018_2023_test, year_count_one, \
        on=["Clean Ward", "End Year for Fill"])
one_entry = one_entry.drop(columns= 'count_by_year')

one_entry

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill
0,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2018.0
1,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2019.0
2,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2020.0
3,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2021.0
4,5,Leslie Hairston,May 1999,"May 15, 2023",None,None,5,1999,2023.0,2022.0
...,...,...,...,...,...,...,...,...,...,...
265,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0
266,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0
267,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0
268,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2023.0


In [99]:
# Code courtesy of: https://stackoverflow.com/questions/38516664/anti-join-pandas

outer_join = alders_2018_2023_test.merge(year_count_over, how = 'outer', indicator = True)
anti_join = outer_join[~(outer_join._merge == 'both')].drop('_merge', axis = 1)

anti_join

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill,count_by_year
0,1st,Proco Joe Moreno,"March 26, 2010","May 20, 2019",Democratic,,1,2010,2019.0,2018.0,NaN
3,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2020.0,NaN
4,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2021.0,NaN
5,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2022.0,NaN
6,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2023.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
325,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2019.0,NaN
326,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0,NaN
327,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0,NaN
328,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0,NaN


6b. Clean up our joined dataset by dropping duplicates, keeping the latter of the two options

In [100]:
filtered_for_problems = pd.merge(alders_2018_2023_test, year_count_over, on=["Clean Ward", "End Year for Fill"])
#filtered_for_problems = filtered_for_problems.reset_index(drop= True)
#filtered_for_problems = filtered_for_problems.sort_values(["Clean Ward", "End Year for Fill"])

filtered_for_problems = filtered_for_problems.drop_duplicates(subset=['Clean Ward', 'End Year for Fill'], keep = 'last')
filtered_for_problems = filtered_for_problems.drop(columns= 'count_by_year')
filtered_for_problems

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill
1,11,Nicole Lee,"March 28, 2022",present,None,None,11,2022,NaN,2022.0
3,12,Anabel Abarca,"December 14, 2022","May 15, 2023",None,None,12,2022,2023.0,2022.0
5,12,Julia Ramirez,"May 15, 2023",present,None,None,12,2023,NaN,2023.0
8,15,Raymond Lopez,"May 18, 2015",present,None,None,15,2015,NaN,2018.0
9,15,Raymond Lopez,"May 18, 2015",present,None,None,15,2015,NaN,2019.0
11,16,Stephanie Coleman,"May 20, 2019",present,None,None,16,2019,NaN,2019.0
13,20,Jeanette Taylor,"May 20, 2019",present,None,None,20,2019,NaN,2019.0
15,22,Michael D. Rodriguez,"May 20, 2019",present,None,None,22,2019,NaN,2019.0
17,23,Silvana Tabares,"June 28, 2018",present,None,None,23,2018,NaN,2018.0
20,33,Rossana Rodriguez-Sanchez,"May 20, 2019",present,None,None,33,2019,NaN,2019.0


6c. Concatenate data

In [102]:
aldermen_checked = pd.concat([filtered_for_problems, anti_join], ignore_index=True)

aldermen_checked = aldermen_checked.sort_values(["Clean Ward", "End Year for Fill"])
aldermen_checked
#aldermen_checked.groupby('Clean Ward').size()

,Ward,Alderperson,Start Date,End Date,Party,Notes,Clean Ward,Start Year,End Year,End Year for Fill,count_by_year
30,1st,Proco Joe Moreno,"March 26, 2010","May 20, 2019",Democratic,,1,2010,2019.0,2018.0,NaN
23,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2019.0,NaN
31,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2020.0,NaN
32,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2021.0,NaN
33,1st,Daniel La Spata,"May 20, 2019",present,Democratic,,1,2019,NaN,2022.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
295,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2019.0,NaN
296,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2020.0,NaN
297,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2021.0,NaN
298,50th,Debra Silverstein,2011,present,N/A,,50,2011,NaN,2022.0,NaN


# Join in Calls per Ward

In [3]:
# Read in calls_money
calls_money = pd.read_table("../data/calls_money", sep= ",", index_col=False)
calls_money["ward"] = pd.to_numeric(calls_money["ward"])

In [13]:
calls_money['year']=calls_money['year'].astype(int)
calls_money['ward']=calls_money['ward'].astype(int)

aldermen_checked['Clean Ward']=aldermen_checked['Clean Ward'].astype(int)
aldermen_checked['End Year for Fill']=aldermen_checked['End Year for Fill'].astype(int)

In [14]:
calls_menu_man = pd.merge(aldermen_checked, calls_money, left_on=["Clean Ward", "End Year for Fill"], right_on=["ward", "year"])

calls_menu_man = calls_menu_man.loc[(calls_menu_man['End Year for Fill'] >= 2018)]

In [15]:
calls_menu_man_clean = calls_menu_man.loc[:, ['Clean Ward', 'Alderperson', 'Start Date', 'End Date', 'End Year for Fill', 'year', 'category', 'calls', 'num_projects', 'total_cost']]

calls_menu_man_clean= calls_menu_man_clean.sort_values(["Clean Ward", "year",])

calls_menu_man_clean = calls_menu_man_clean.reset_index(drop=True)

In [16]:
calls_menu_man_clean.to_csv("../data/calls_menu_man.csv")